In [1]:
import pandas as pd

In [2]:
rfg_seafood = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Rfg Seafood_POS_2020.xlsx')
rfg_seafood2 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Rfg Seafood_POS_2021.xlsx')
rfg_seafood3 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Rfg Seafood_POS_2022.xlsx')
rfg_seafood4 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Rfg Seafood_POS_2023.xlsx')
rfg_seafood5 = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Data/Data/Rfg Seafood_POS_2024.xlsx')
product_attribute = pd.read_excel('/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/Conagra Cleaned Data/Product Attributes.xlsx')


In [3]:
# Imputation
def imputation_process(row):
    row['Incremental Units'] = row['Unit Sales'] - row['Base Unit Sales']
    row['Incremental Volume'] = row['Volume Sales'] - row['Base Volume Sales']
    row['Incremental Dollars'] = row['Dollar Sales'] - row['Base Dollar Sales']
    return row

# Split Geography column and only save its location
def extract_location(col):
    return col.split(' - ')[0]

# convert time into year month and day
def parse_time(col):
    # Year, month, day
    sep_data = col.split('-')
    year = int('20' + sep_data[2])
    month = int(sep_data[0][-2:])
    day = int(sep_data[1])
    # Quarter of given date
    quarter = (month - 1) // 3 + 1
    return pd.Series([year, quarter, month], index = ['Year', 'Quarter', 'Month'])

# Split product column and only save company name and product name
def parse_product(col):  
    return col.split(' - ')[0]


# Based on length of company name, select product name
def extract_product_name(full_string, brand_name):
    brand_name_length = len(brand_name)
    words = full_string[brand_name_length:].strip()
    return words

# # Arrange columns order
def change_column_order(df):
    final_order = [
        'Geography', 'Year', 'Quarter', 'Month', 'Season', 'Manufacturer Name',
        'Brand Name', 'Product Name', 'Flavor / Scent', 'Meat Source',
        'Unit Sales', 'Volume Sales', 'Dollar Sales', 'Price per Unit', 
        'Price per Volume', 'Base Unit Sales', 'Base Volume Sales', 'Base Dollar Sales',
        'Incremental Units', 'Incremental Volume', 'Incremental Dollars', 
        'ACV Weighted Distribution'
    ]
    return df[final_order]

def merge_content(df1, df2):
    combined = df1.merge(df2, how = 'left', on = 'Product')
    return combined

def map_season(month):
    # Northern Hemisphere standard mapping
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'


# Full tranform process
def transform_data(df, df2):
    table1 = merge_content(df, df2)
    # Used customized function
    table1['Geography'] = table1['Geography'].map(extract_location)
    table1[['Year', 'Quarter', 'Month']] = table1['Time'].apply(parse_time)
    table1['Product'] = table1['Product'].map(parse_product)
    table1['Product Name'] = table1.apply(lambda row: extract_product_name(row['Product'], row['Brand Name']), axis=1)
    #df['Product Name'] = df.apply(extract_product_name(df['Product'], df['Brand']), axis = 1)
    
    # Mapping quarter to seasonality
    table1['Season'] = table1['Month'].apply(map_season)
    # Change column order
    transformed = change_column_order(table1).apply(imputation_process, axis = 1)
    return transformed

In [4]:
rfg_seafood_2020 = transform_data(rfg_seafood, product_attribute)
rfg_seafood_2021 = transform_data(rfg_seafood2, product_attribute)
rfg_seafood_2022 = transform_data(rfg_seafood3, product_attribute)
rfg_seafood_2023 = transform_data(rfg_seafood4, product_attribute)
rfg_seafood_2024 = transform_data(rfg_seafood5, product_attribute)

In [5]:
rfg_seafood_2020.head()

,Geography,Year,Quarter,Brand,Product Name,Unit Sales,Volume Sales,Dollar Sales,Price per Unit,Price per Volume,Base Unit Sales,Base Volume Sales,Base Dollar Sales,Incremental Units,Incremental Volume,Incremental Dollars,ACV Weighted Distribution
0,Total US,2020,1,ALTMANS,REFRIGERATED REGULAR HERRING 13 OZ,287.743122,233.791287,1579.709742,5.490000,6.756923,282.998601,229.936363,1553.615193,4.744522,3.854924,26.094549,0.800326
1,Total US,2020,1,ALTMANS,REFRIGERATED REGULAR HERRING 13 OZ,237.370888,192.863847,1303.166178,5.490000,6.756923,229.191303,186.217934,1258.269686,8.179585,6.645913,44.896492,0.831006
2,Total US,2020,1,ALTMANS,REFRIGERATED REGULAR HERRING BIT 26 OZ,256.708530,417.151362,2051.101156,7.990000,4.916923,255.242831,414.769601,2039.370679,1.465699,2.381761,11.730478,0.857692
3,Total US,2020,1,ALTMANS,REFRIGERATED REGULAR HERRING CREAM SAUCE 13 OZ,211.093960,171.513843,1158.905843,5.490000,6.756923,203.635826,165.454108,1118.028030,7.458135,6.059734,40.877812,0.803703
4,Total US,2020,1,SKANSEN,REFRIGERATED REGULAR HERRING TIDBIT SOUR CREAM...,78.995556,39.497778,399.641907,5.059043,10.118086,78.995556,39.497778,399.641907,0.000000,0.000000,0.000000,0.130395


In [6]:
# Combined & export
full_rfg_seafood = pd.concat([rfg_seafood_2020, rfg_seafood_2021, rfg_seafood_2022, rfg_seafood_2023, rfg_seafood_2024])
output_file = '/Users/fuqiaozhi/Desktop/UTD Semesters/Spring 2024/BUAN 6337/full_rfg_seafood.csv'
full_rfg_seafood.to_csv(output_file, index=False)